In [1]:
# Muhammad Fauzan Nur'ilham
# 1103204085
# Machine Learning
# TK-44-G4

In [2]:
# Mengimpor pustaka yang diperlukan dari PyTorch untuk membangun dan melatih jaringan saraf
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Menyiapkan komponen-komponen yang diperlukan untuk membangun dan melatih jaringan saraf menggunakan PyTorch
# - torch: Pustaka PyTorch untuk komputasi tensor
# - torch.nn: Modul jaringan saraf PyTorch untuk membangun dan melatih jaringan saraf
# - torch.optim: Modul optimisasi PyTorch untuk menentukan dan menggunakan algoritma optimisasi
# - torchvision: Pustaka PyTorch untuk menangani dataset dan model penglihatan komputer
# - torchvision.transforms: Modul PyTorch untuk mendefinisikan transformasi gambar
# - torch.utils.data: Modul PyTorch untuk menangani pengambilan data dan pembuatan batch

In [3]:
# Memilih perangkat (device) yang akan digunakan, jika CUDA (GPU) tersedia, gunakan CUDA, jika tidak, gunakan CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Mendefinisikan kelas AlexNetMNIST sebagai turunan dari nn.Module
class AlexNetMNIST(nn.Module):
    def __init__(self):
        super(AlexNetMNIST, self).__init__()

        # Definisi layer-layer dari arsitektur AlexNet
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10),  # 10 classes for MNIST
        )

    # Definisi langkah-langkah forward propagation
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [5]:
# Menentukan serangkaian transformasi untuk preprocessing data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Mengubah ukuran gambar menjadi (224, 224)
    transforms.ToTensor(),  # Mengubah gambar menjadi tensor PyTorch
])

In [6]:
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 132334656.11it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 15884565.15it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 109603819.34it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 4156781.32it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [7]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [8]:
# Membuat instance dari kelas AlexNetMNIST dan memindahkannya ke perangkat yang telah dipilih
model = AlexNetMNIST().to(device)

# Menentukan fungsi loss (CrossEntropyLoss) untuk klasifikasi multikelas
criterion = nn.CrossEntropyLoss()

# Menentukan algoritma optimisasi (Adam) dengan learning rate sebesar 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# Menentukan jumlah epoch
num_epochs = 5

# Melakukan iterasi untuk setiap epoch
for epoch in range(num_epochs):
    # Mengaktifkan mode pelatihan pada model
    model.train()

    # Menginisialisasi total_loss untuk setiap epoch
    total_loss = 0.0

    # Iterasi untuk setiap batch pada data pelatihan
    for inputs, labels in train_loader:
        # Memindahkan data ke perangkat yang telah ditentukan sebelumnya (cuda/cpu)
        inputs, labels = inputs.to(device), labels.to(device)

        # Mengatur gradien parameter model menjadi 0
        optimizer.zero_grad()

        # Melakukan forward pass
        outputs = model(inputs)

        # Menghitung nilai fungsi loss
        loss = criterion(outputs, labels)

        # Melakukan backward pass dan optimasi parameter model
        loss.backward()
        optimizer.step()

        # Menambahkan nilai loss pada total_loss
        total_loss += loss.item()

    # Menampilkan informasi hasil pelatihan pada setiap epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/5, Loss: 0.27832232897571013
Epoch 2/5, Loss: 0.0874824858247501
Epoch 3/5, Loss: 0.07202413928474603
Epoch 4/5, Loss: 0.06031352254287921
Epoch 5/5, Loss: 0.05407678246641952


In [10]:
# Menentukan jumlah epoch
num_epochs = 5

# Melakukan iterasi untuk setiap epoch
for epoch in range(num_epochs):
    # Mengaktifkan mode pelatihan pada model
    model.train()

    # Menginisialisasi total_loss untuk setiap epoch
    total_loss = 0.0

    # Iterasi untuk setiap batch pada data pelatihan
    for inputs, labels in train_loader:
        # Memindahkan data ke perangkat yang telah ditentukan sebelumnya (cuda/cpu)
        inputs, labels = inputs.to(device), labels.to(device)

        # Mengatur gradien parameter model menjadi 0
        optimizer.zero_grad()

        # Melakukan forward pass
        outputs = model(inputs)

        # Menghitung nilai fungsi loss
        loss = criterion(outputs, labels)

        # Melakukan backward pass dan optimasi parameter model
        loss.backward()
        optimizer.step()

        # Menambahkan nilai loss pada total_loss
        total_loss += loss.item()

    # Menampilkan informasi hasil pelatihan pada setiap epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/5, Loss: 0.05119844949766082
Epoch 2/5, Loss: 0.04783052914441555
Epoch 3/5, Loss: 0.045192258018747544
Epoch 4/5, Loss: 0.039444816615593375
Epoch 5/5, Loss: 0.039022260581911584
